In [1]:
import requests
from requests import Response
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res:Response = requests.get(youbike_url)

if res.status_code == 200:
    print("下載成功")
else:
    print("下載失敗")


from pydantic import BaseModel,Field,field_validator,RootModel
class Site (BaseModel):
    站名:str = Field(alias='sna') 
    行政區域:str = Field(alias='sarea')
    時間:str = Field(alias='mday')
    位置:str = Field(alias='ar')
    狀態:bool = Field(alias='act')
    總數量:int = Field(alias='total')
    可借:int = Field(alias='available_rent_bikes')
    緯度:float = Field(alias='latitude')
    經度:float = Field(alias='longitude')     
    可還:int = Field(alias='available_return_bikes')

    @field_validator('站名',mode='before')
    @classmethod
    def get_name(cls,value:str) ->str:
         return value.split('_')[-1]   

class Youbick(RootModel):
    root:list[Site]


data = Youbick.model_validate_json(res.text)
all_sites = data.model_dump()


import pandas as pd
df = pd.DataFrame(all_sites)
df

下載成功


,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
0,捷運科技大樓站,大安區,2024-06-17 19:27:20,復興南路二段235號前,True,28,4,25.02605,121.54360,24
1,復興南路二段273號前,大安區,2024-06-17 19:24:21,復興南路二段273號西側,True,21,8,25.02565,121.54357,13
2,國北教大實小東側門,大安區,2024-06-17 19:26:22,和平東路二段96巷7號,True,16,3,25.02429,121.54124,13
3,和平公園東側,大安區,2024-06-17 19:26:21,和平東路二段118巷33號,True,11,10,25.02351,121.54282,1
4,辛亥復興路口西北側,大安區,2024-06-17 19:27:20,復興南路二段368號,True,16,6,25.02153,121.54299,10
...,...,...,...,...,...,...,...,...,...,...
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:27:14,臺大圖書館西南側,True,30,0,25.01690,121.54031,30
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:21:21,臺大霖澤館南側,True,20,1,25.01995,121.54347,19
1420,臺大獸醫館南側,臺大公館校區,2024-06-17 19:13:50,臺大獸醫系館南側,True,24,1,25.01791,121.54242,23
1421,臺大新體育館東南側,臺大公館校區,2024-06-17 19:27:19,臺大體育館東側,True,40,9,25.02112,121.53591,31


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423 entries, 0 to 1422
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   站名      1423 non-null   object 
 1   行政區域    1423 non-null   object 
 2   時間      1423 non-null   object 
 3   位置      1423 non-null   object 
 4   狀態      1423 non-null   bool   
 5   總數量     1423 non-null   int64  
 6   可借      1423 non-null   int64  
 7   緯度      1423 non-null   float64
 8   經度      1423 non-null   float64
 9   可還      1423 non-null   int64  
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 101.6+ KB


In [2]:
df[df['可借'] <= 3]

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
2,國北教大實小東側門,大安區,2024-06-17 19:26:22,和平東路二段96巷7號,True,16,3,25.024290,121.541240,13
16,和平臥龍街口,大安區,2024-06-17 19:25:22,臥龍街1號,True,15,2,25.024461,121.546705,13
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,False,5,0,25.020930,121.533100,5
20,第二學生活動中心,大安區,2024-06-17 19:25:22,羅斯福路四段107號(西南側),True,25,1,25.012440,121.536610,24
21,臺灣科技大學正門,大安區,2024-06-17 19:06:22,基隆路四段43號(臺灣科技大學正門旁小側門靠田徑場),True,50,2,25.014510,121.541420,48
...,...,...,...,...,...,...,...,...,...,...
1416,臺大電機二館東南側,臺大公館校區,2024-06-17 19:25:22,臺大電機二館南側,True,14,3,25.018570,121.542470,11
1417,臺大圖資系館北側,臺大公館校區,2024-06-17 19:25:22,臺大圖書資訊館北側,True,25,1,25.018040,121.539280,24
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:27:14,臺大圖書館西南側,True,30,0,25.016900,121.540310,30
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:21:21,臺大霖澤館南側,True,20,1,25.019950,121.543470,19


In [4]:
import numpy as np

less3_df = df.query('可借<=3')
less3_df.shape

(446, 10)

In [5]:
df2 = df.query('可借<=3 and 可還>=30')
df2

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
21,臺灣科技大學正門,大安區,2024-06-17 19:06:22,基隆路四段43號(臺灣科技大學正門旁小側門靠田徑場),True,50,2,25.01451,121.54142,48
22,臺灣科技大學側門,大安區,2024-06-17 19:27:20,基隆路四段73巷與基隆路口,True,99,1,25.01295,121.53973,98
47,嘉興公園,大安區,2024-06-17 19:24:20,樂業街101巷3號西北側,True,44,3,25.02100,121.55211,36
49,捷運六張犁站,大安區,2024-06-17 19:27:20,和平東路三段410號,True,64,2,25.02397,121.55266,60
68,敦化基隆路口,大安區,2024-06-17 19:17:30,敦化南路二段218號,True,38,0,25.02215,121.54839,36
...,...,...,...,...,...,...,...,...,...,...
1399,臺大社科院西側,臺大公館校區,2024-06-17 19:24:20,臺大社科院西側,True,42,2,25.02053,121.54145,40
1400,臺大社會系館南側,臺大公館校區,2024-06-17 19:24:15,臺大社會及社工館南側,True,39,1,25.01987,121.54175,38
1405,臺大生命科學館西北側,臺大公館校區,2024-06-17 19:24:15,臺大動物博物館西側,True,30,0,25.01548,121.53825,30
1406,臺大第一活動中心西南側,臺大公館校區,2024-06-17 19:26:22,第一學生活動中心東南側,True,40,2,25.01761,121.53995,38


In [6]:
df2 = df.query('可借<=3 or 可還<=3')
df2

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
2,國北教大實小東側門,大安區,2024-06-17 19:26:22,和平東路二段96巷7號,True,16,3,25.02429,121.54124,13
3,和平公園東側,大安區,2024-06-17 19:26:21,和平東路二段118巷33號,True,11,10,25.02351,121.54282,1
5,復興南路二段280號前,大安區,2024-06-17 19:05:23,復興南路二段280號,True,11,10,25.02429,121.54328,1
6,復興南路二段340巷口,大安區,2024-06-17 19:25:22,復興南路二段342號,True,11,8,25.02253,121.54326,3
7,新生南路三段52號前,大安區,2024-06-17 19:17:30,新生南路三段52號,True,17,17,25.02112,121.53407,0
...,...,...,...,...,...,...,...,...,...,...
1416,臺大電機二館東南側,臺大公館校區,2024-06-17 19:25:22,臺大電機二館南側,True,14,3,25.01857,121.54247,11
1417,臺大圖資系館北側,臺大公館校區,2024-06-17 19:25:22,臺大圖書資訊館北側,True,25,1,25.01804,121.53928,24
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:27:14,臺大圖書館西南側,True,30,0,25.01690,121.54031,30
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:21:21,臺大霖澤館南側,True,20,1,25.01995,121.54347,19


In [ ]:
df3 = df.query('行政區域=="大安區" and 可借==0') 
df3

In [9]:
建立變數
n=0
df3 = df.query('行政區域=="大安區" and 可借==@n') 
df3

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,False,5,0,25.020930,121.53310,5
31,臺大動物醫院前,大安區,2024-06-17 19:13:15,基隆路三段155巷/基隆路四段口,True,18,0,25.015760,121.54291,18
41,和平實小東北側,大安區,2024-06-17 19:27:20,敦南街73號,True,13,0,25.021641,121.54704,13
51,法治公園,大安區,2024-06-17 19:26:22,通化街189號西側,True,15,0,25.026940,121.55325,10
58,捷運古亭站(5號出口),大安區,2024-06-17 19:27:20,和平東路一段19號前,True,14,0,25.027150,121.52293,14
68,敦化基隆路口,大安區,2024-06-17 19:17:30,敦化南路二段218號,True,38,0,25.022150,121.54839,36
69,安和路二段181巷口,大安區,2024-06-17 19:16:15,安和路二段185號,True,17,0,25.027210,121.54988,17
76,復興南路1段340巷口,大安區,2024-06-17 19:27:14,復興南路一段信義路三段147巷45弄口,True,50,0,25.037010,121.54348,50
88,昌隆公園,大安區,2024-06-17 19:26:22,市民大道三段206號西側,True,16,0,25.044480,121.53946,16
98,瑠公公園,大安區,2024-03-30 08:59:14,忠孝東路四段49巷4弄/大安路口,False,20,0,25.042010,121.54612,20
